## 1日目
型クラス初級講座

### Equal

In [2]:
import scalaz._, Scalaz._

import scalaz._, Scalaz._

In [3]:
1 === 1

res2: Boolean = true

In [3]:
1 === "foo"

cmd3.sc:1: type mismatch;
 found   : String("foo")
 required: Int
val res3 = 1 === "foo"
                 ^

: 

In [4]:
1 == "foo"

res3: Boolean = false

In [5]:
1.some =/= 2.some

res4: Boolean = true

In [6]:
1 assert_=== 2

: 

In [7]:
1 != 2 && false

res6: Boolean = false

In [7]:
1 /== 2 && false

cmd7.sc:1: value && is not a member of Int
val res7 = 1 /== 2 && false
                   ^

: 

In [8]:
1 =/= 2 && false

res7: Boolean = false

* `/==` はカス。`=/=` を使う事

### Order
* Scala Ord
* Scalaz Order

In [9]:
1 > 2.0

res8: Boolean = false

In [9]:
1 gt 2.0

cmd9.sc:1: type mismatch;
 found   : Double(2.0)
 required: Int
val res9 = 1 gt 2.0
                ^

: 

In [10]:
1.0 ?|? 2.0

res9: Ordering = LT

In [11]:
1.0 max 2.0

res10: Double = 2.0

* IntとDoubleの比較ならコンパイルエラーにならなくても良いような気がするけど、 `1 = 1.0` じゃないからってことかな

### Show
* Scala: Show
* Scalaz: Show

In [12]:
3.show

res11: Cord = 3

In [13]:
3.shows

res12: String = "3"

In [14]:
"hello".println

"hello"


* ( ..)φ CordはScalazの潜在的に長い可能性のある文字列を保持できる純粋関数型データ構造
* `final case class Cord(self: FingerTree[Int, String])`

### Read
* 対応するScalazの型クラスなし?

### Enum
* Scala: Enum
* Scalaz: Enum

In [15]:
'a' to 'e'

res14: collection.immutable.NumericRange.Inclusive[Char] = NumericRange('a', 'b', 'c', 'd', 'e')

In [16]:
'a' |-> 'e'

res15: List[Char] = List('a', 'b', 'c', 'd', 'e')

In [17]:
3 |=> 5

res16: EphemeralStream[Int] = scalaz.EphemeralStream$$anon$5@101bf6b1

In [18]:
'B'.succ

res17: Char = 'C'

### Bounded
Bounded型クラスのインスタンスは上限と下限値を持ち、minBoundとmaxBoundで調べる事ができる
* Scala:  Bounded
* Scalaz: Enum

In [19]:
implicitly[Enum[Char]].min

res18: Option[Char] = Some(' ')

In [20]:
implicitly[Enum[Char]].max

res19: Option[Char] = Some('￿')

In [22]:
implicitly[Enum[Double]].max

cmd22.sc:1: could not find implicit value for parameter e: scalaz.Enum[Double]
val res22 = implicitly[Enum[Double]].max
                      ^

: 

In [21]:
implicitly[Enum[Int]].min

res20: Option[Int] = Some(-2147483648)

In [22]:
implicitly[Enum[(Boolean, Int, Char)]].max

cmd22.sc:1: could not find implicit value for parameter e: scalaz.Enum[(Boolean, Int, Char)]
val res22 = implicitly[Enum[(Boolean, Int, Char)]].max
                      ^

: 

* `Enum[Double]` もないんだけど
* `def implicitly[T](implicit e: T): T = e` 型レベルの計算の結果を確認するのに使われるテクニックらしい

In [27]:
implicitly[Int <:< AnyVal]

res26: Int <:< AnyVal = <function1>

### Num
* Num, FLoating, Integral に対応する型クラスはなし?

### 型クラス中級講座
* 信号の型クラスを作る

In [28]:
sealed trait TrafficLight
case object Red extends TrafficLight
case object Yellow extends TrafficLight
case object Green extends TrafficLight

defined trait TrafficLight
defined object Red
defined object Yellow
defined object Green

* この書き方は、 algebraic data typeを書くときのパターンらしい
* sealed があると、match で case を書き漏らしたときにコンパイルエラーが出るらしい

In [29]:
implicit val TrafficLightEqual: Equal[TrafficLight] = Equal.equal(_ == _)

TrafficLightEqual: Equal[TrafficLight] = scalaz.Equal$$anon$7@5cd0a32f

In [29]:
Red === Yellow

cmd29.sc:1: value === is not a member of object cmd29Wrapper.this.cmd27.wrapper.Red
val res29 = Red === Yellow
                ^

: 

In [33]:
case class TrafficLight(name: String)
val red = TrafficLight("red")
val yellow = TrafficLight("yellow")
val green = TrafficLight("green")
implicit val TrafficLightEqual: Equal[TrafficLight] = Equal.equal(_ == _)
red === yellow

defined class TrafficLight
red: wrapper.wrapper.TrafficLight = TrafficLight("red")
yellow: wrapper.wrapper.TrafficLight = TrafficLight("yellow")
green: wrapper.wrapper.TrafficLight = TrafficLight("green")
TrafficLightEqual: Equal[wrapper.wrapper.TrafficLight] = scalaz.Equal$$anon$7@355cad15
res32_5: Boolean = false

In [32]:
scalaz.Equal

res31: Equal.type = scalaz.Equal$@6af29ce1

* 何で最初のパターンで上手くいかないのかな?
* 最初のパターンの方が綺麗なのでそれでやりたいんだけど

### Yes と No の型クラス

In [34]:
trait CanTruthy[A] { self =>
    def truthys(a: A): Boolean
}
object CanTruthy {
    def apply[A](implicit ev: CanTruthy[A]): CanTruthy[A] = ev
    def truthys[A](f: A => Boolean): CanTruthy[A] = new CanTruthy[A] {
        def truthys(a: A): Boolean = f(a)
    }
}
trait CanTruthyOps[A] {
    def self: A
    implicit def F: CanTruthy[A]
    final def truthy: Boolean = F.truthys(self)
}
object ToCanIsTruthyOps {
    implicit def toCanIsTruthyOps[A](v: A)(implicit ev: CanTruthy[A]) =
    new CanTruthyOps[A] {
        def self = v
        implicit def F: CanTruthy[A] = ev
    }
}

defined trait CanTruthy
defined object CanTruthy
defined trait CanTruthyOps
defined object ToCanIsTruthyOps

In [35]:
import ToCanIsTruthyOps._

import ToCanIsTruthyOps._

In [36]:
implicit val intCanTruthy: CanTruthy[Int] = CanTruthy.truthys({
    case 0 => false
    case _ => true
})

intCanTruthy: CanTruthy[Int] = $sess.cmd33Wrapper$Helper$CanTruthy$$anon$1@62677b0d

In [37]:
10.truthy
0.truthy

res36_0: Boolean = true
res36_1: Boolean = false

In [38]:
implicit def listCanTruthy[A]: CanTruthy[List[A]] = CanTruthy.truthys({
    case Nil => false
    case _ => true
})

defined function listCanTruthy

In [39]:
Nil.truthy

cmd39.sc:1: value truthy is not a member of object Nil
val res39 = Nil.truthy
                ^

: 

In [40]:
implicit val nilCanTruthy: CanTruthy[scala.collection.immutable.Nil.type] = CanTruthy.truthys({
    _ => false
})

nilCanTruthy: CanTruthy[Nil.type] = $sess.cmd33Wrapper$Helper$CanTruthy$$anon$1@bbca42b

In [41]:
Nil.truthy

res40: Boolean = false

In [42]:
implicit val booleanCanTruthy: CanTruthy[Boolean] = CanTruthy.truthys(identity)

booleanCanTruthy: CanTruthy[Boolean] = $sess.cmd33Wrapper$Helper$CanTruthy$$anon$1@5ddddc8f

In [43]:
false.truthy

res42: Boolean = false

In [44]:
def truthyIf[A: CanTruthy, B, C](cond: A)(ifyes: => B)(ifno: => C) =
  if (cond.truthy) ifyes
  else ifno

defined function truthyIf

In [45]:
truthyIf (Nil) {"YEAH!"} {"NO!"}

res44: Any = NO!

In [46]:
truthyIf (2 :: 3 :: 4 :: Nil) {"YEAH!"} {"NO!"}

res45: Any = YEAH!

In [47]:
truthyIf (true) {"YEAH!"} {"NO!"}

res46: Any = YEAH!

* テキストに書いてある「不変な型パラメータのせい」というのがあんまり理解できていない :thinkingface: